Business Objectives:
Which product is the most popular by volume?
Which product is the most popular amount spent?

What we want to discover:
1. Find out product similarity based on product description (via cosign similarity)
2. Make reccomendations using Matrix Vectorization

# 1.1 Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib as lp

# 1.2 Load the Data

In [4]:
data = pd.read_csv('1533148922_PBL 5 recommendation data.csv', encoding="latin-1")
data.head(5)
#at default encoding set at utf8, if you get an error at utf8, change to latin-1(depends on how file is saved)

/Users/alisonbunce/anaconda3/envs/DeepLearning/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (20,33,73,106,158) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Customers.id,Customers.fname,Customers.lname,Customers.company,Customers.create_date,Customers.status,Customers.mailing,Customers.reminders,Customers.tax_exempt,Customers.account_id,...,Products.google_shopping_label,Products.product_option,Products.size,Products.material,Products.arm_style,Products.leg_style,Products.seat_size,Products.family_id,Products.saved_status,Products.freight_cost
0,797,Christy,Dill,Company0,1426018724,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PF61071,0.0,NaN
1,3,John,Smith,Company1,1386089139,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PF02132,NaN,NaN
2,3,John,Smith,Company1,1386089139,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2 x Extra large,Nitrile,NaN,NaN,NaN,PF00342,0.0,NaN
3,4,James,Anderson,NaN,1386780263,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PF04970,NaN,NaN
4,5,Abraham,Pollak,Company3,1386861599,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PF03045,NaN,NaN


In [5]:
data.info()
#we have 4194 rows and 181 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4194 entries, 0 to 4193
Columns: 181 entries, Customers.id to Products.freight_cost
dtypes: float64(98), int64(10), object(73)
memory usage: 5.8+ MB


In [9]:
data.nunique()

Customers.id             3054
Customers.fname          1631
Customers.lname          2557
Customers.company         425
Customers.create_date    3054
                         ... 
Products.leg_style          0
Products.seat_size         13
Products.family_id       1002
Products.saved_status       2
Products.freight_cost       0
Length: 181, dtype: int64

In [10]:
#convert to a dataframe so easier to read
pd.DataFrame(data.nunique())

,0
Customers.id,3054
Customers.fname,1631
Customers.lname,2557
Customers.company,425
Customers.create_date,3054
...,...
Products.leg_style,0
Products.seat_size,13
Products.family_id,1002
Products.saved_status,2


In [11]:
#save this dataframe for reference
df = pd.DataFrame(data.nunique())

# 2. Understand the Data

Let's determine the most popular product by volume

In [13]:
#which column reflects the volume
data.columns

Index(['Customers.id', 'Customers.fname', 'Customers.lname',
       'Customers.company', 'Customers.create_date', 'Customers.status',
       'Customers.mailing', 'Customers.reminders', 'Customers.tax_exempt',
       'Customers.account_id',
       ...
       'Products.google_shopping_label', 'Products.product_option',
       'Products.size', 'Products.material', 'Products.arm_style',
       'Products.leg_style', 'Products.seat_size', 'Products.family_id',
       'Products.saved_status', 'Products.freight_cost'],
      dtype='object', length=181)

In [14]:
print(data.columns)

Index(['Customers.id', 'Customers.fname', 'Customers.lname',
       'Customers.company', 'Customers.create_date', 'Customers.status',
       'Customers.mailing', 'Customers.reminders', 'Customers.tax_exempt',
       'Customers.account_id',
       ...
       'Products.google_shopping_label', 'Products.product_option',
       'Products.size', 'Products.material', 'Products.arm_style',
       'Products.leg_style', 'Products.seat_size', 'Products.family_id',
       'Products.saved_status', 'Products.freight_cost'],
      dtype='object', length=181)


In [15]:
#list of all the columns in our dataframe
print(list(data.columns))

['Customers.id', 'Customers.fname', 'Customers.lname', 'Customers.company', 'Customers.create_date', 'Customers.status', 'Customers.mailing', 'Customers.reminders', 'Customers.tax_exempt', 'Customers.account_id', 'Customers.sales_rep', 'Customers.rewards', 'Customers.profile_id', 'Customers.last_modified', 'Customers.customer_type', 'Orders.id', 'Orders.customer_id', 'Orders.fname', 'Orders.lname', 'Orders.company', 'Orders.order_number', 'Orders.reorder_id', 'Orders.external_source', 'Orders.external_id', 'Orders.currency', 'Orders.sales_rep', 'Orders.subtotal', 'Orders.tax', 'Orders.shipping', 'Orders.coupon_id', 'Orders.coupon_amount', 'Orders.gift_id', 'Orders.gift_amount', 'Orders.fee_name', 'Orders.fee_amount', 'Orders.discount_name', 'Orders.discount_amount', 'Orders.total', 'Orders.balance_due', 'Orders.shipping_carrier', 'Orders.shipping_method', 'Orders.shipping_trans', 'Orders.shipping_flags', 'Orders.weight', 'Orders.tracking', 'Orders.payment_status', 'Orders.payment_date'

## 3. Determine relevant columns

Now let's determine which column will reflect volume. This data is based on quantities ordered, so let's use quantity.

In [16]:
'qty' in data.columns

False

In [17]:
'Qty' in data.columns

False

In [21]:
'Order_Items.qty' in data.columns

True

In [22]:
'Products.name' in data.columns

True

## 4. Create a new dataframe for relevant columns

In [26]:
#create new dataframe to see the total list of products and total quantity of products ordered
dataPQ = data[['Order_Items.qty', 'Products.name', 'Order_Items.price']]

In [27]:
#call the dataframe
dataPQ

,Order_Items.qty,Products.name,Order_Items.price
0,1,"Basic Steel Rollators,Green",57.20
1,4,Urinary Drain Bags,5.00
2,1,"SensiCare Nitrile Exam Gloves,Blue,XX-Large",68.78
3,1,Basket for 2-Button Walkers,19.56
4,1,TENS 3000 Analog Unit,36.05
...,...,...,...
4189,1,"Emesis Bags,Blue,36.000 OZ",14.85
4190,1,Bed Assist Bar,39.99
4191,1,Spitfire EX Compact Travel Power Mobility Scoo...,1099.00
4192,1,"Standard Bariatric Heavy Duty Rollator,Burgund...",129.99


## 5. Test our Recommendations w/ the orig data

Analysis note: we want to determine the max prod quantity ordered, (aka the highest volume product). To do this, we group the products together and find the total sum of the quantites for each product.

In [30]:
#list
dataPQ.groupby('Products.name')['Order_Items.qty'].sum()

Products.name
1 Box Of Medline Commode Liners with Absorbent Pad                                      2
1-Ply Blockade AngelStat Surgical Gowns,Ceil Blue,Extra Large                           1
1/2" self-adhesive hook material, 25 yard dispenser box, white                          1
1/2" self-adhesive loop material, 25 yard dispenser box, white                          1
100% Cotton Unisex Reversible Hyperbaric Scrub Tops,Misty Green,MZS                     3
                                                                                       ..
Youth-Sized Walker Platform Attachment                                                  1
Zip-Style Biohazard Specimen Bags,Clear                                                 1
iWalker Euro Style Rollator                                                             1
sEMG - MyoTrac Home Trainer with Myoscan Active Sensor and extender cable, 1-channel    1
sEMG - single use uni-gel electrodes only, case of 100                                

tells us what quanitity of each product was ordered

In [32]:
#now lets sort above by values
dataPQ.groupby('Products.name')['Order_Items.qty'].sum().sort_values(ascending=False)

Products.name
Disposable Emergency Blanket,Not Applicable                   396
MoliCare Disposable Super Plus Briefs,27"-47"                 228
2-Ply Tissue/Poly Professional Towels,Not Applicable          220
Sanitary Pads with Adhesive & Wings                           133
Emesis Bags,Blue,36.000 OZ                                    129
                                                             ... 
Dynarex 4234, Stool Container - Polystyrene    100/cs           1
Rigid Post-Op Shoes,Navy Blue,Medium                            1
Dynarex 3596, Athletic Tape, 1&frac12;" x 15 yds., 4/8/Cs       1
Dynarex 3232, Gauze Sponge N/S 3x3  8 Ply - 20/200 (4M)/Cs      1
sEMG - single use uni-gel electrodes only, case of 100          1
Name: Order_Items.qty, Length: 1391, dtype: int64

our most popular product is at the top. Based on the total number purchased, the Disposable Emergency Blanket is the most purchased item

## 6. Additional Analysis to answer our business objective questions

Now let's find out the most popular product based on total $ amount (sales). So we want to pull the highest number of the total items price column to determine this

In [34]:
dataPQ.groupby('Products.name')['Order_Items.price'].sum().sort_values(ascending=False)

Products.name
Aluminum Transport Chair with 12" Wheels,Red,F: 8   R: 12     8333.02
Aluminum Transport Chair with 12" Wheels,Blue,F: 8   R: 12    8304.39
K1 Basic Extra-Wide Wheelchairs                               6631.76
Extra-Wide Wheelchairs                                        4939.99
K1 Basic Wheelchairs                                          4657.00
                                                               ...   
OR Sterile Specimen Containers,120.0 ML                          2.84
Double-Sided Sterile Shave Prep Razor                            2.52
Sterile Eye Pads                                                 2.37
Perineal Irrigation Bottle,8.000 OZ                              2.31
Paper Measuring Tapes,72.00 IN                                   1.12
Name: Order_Items.price, Length: 1391, dtype: float64

This tells us that prouduct that genereted the highest sales was, Aluminum Transport Chair with 12" Wheels,Red

Now let's find out how many people ordered each product from these two columns

In [35]:
#size tells us how many people ordered
#sum tells us how mamy proucts sold
dataPQ.groupby('Products.name').agg({'Order_Items.qty':[np.size, np.sum]})

Order_Items.qty    
                                                              size sum
Products.name                                                         
1 Box Of Medline Commode Liners with Absorbent Pad               1   2
1-Ply Blockade AngelStat Surgical Gowns,Ceil Bl...               1   1
1/2" self-adhesive hook material, 25 yard dispe...               1   1
1/2" self-adhesive loop material, 25 yard dispe...               1   1
100% Cotton Unisex Reversible Hyperbaric Scrub ...               2   3
...                                                            ...  ..
Youth-Sized Walker Platform Attachment                           1   1
Zip-Style Biohazard Specimen Bags,Clear                          1   1
iWalker Euro Style Rollator                                      1   1
sEMG - MyoTrac Home Trainer with Myoscan Active...               1   1
sEMG - single use uni-gel electrodes only, case...               1   1

[1391 rows x 2 columns]

Let's determine what products were purchased from the most people

In [40]:
#we want to see the amount of people that bought certain products. show us the most people that bought the most prod
dataPQ.groupby('Products.name').agg({'Order_Items.qty':[np.size, np.sum]}).sort_values(by=('Order_Items.qty', 'size'), ascending=False)

Order_Items.qty     
                                                              size  sum
Products.name                                                          
Emesis Bags,Blue,36.000 OZ                                     103  129
MoliCare Disposable Super Plus Briefs,27"-47"                   71  228
Aluminum Transport Chair with 12" Wheels,Blue,F...              67   68
Aluminum Transport Chair with 12" Wheels,Red,F:...              67   68
K1 Basic Extra-Wide Wheelchairs                                 54   58
...                                                            ...  ...
Foam Side Rail Bumper Pads                                       1    1
Folding Bedside Commode with Bucket and Splash ...               1    1
Folding Blind Cane with Wrist Strap                              1    1
Folding Exercise Peddler with Digital Display, ...               1    1
sEMG - single use uni-gel electrodes only, case...               1    1

[1391 rows x 2 columns]

Emesis Bags,Blue,36.000 OZ	were the most frequently ordered by the most number of people. Bought by the most amount of people, sorted by the biggest amount sold.

Analysis: So based on this analysis we can recommend these best selling products specifically because we don't yet know the customer/user's preferences